In [1]:
# load relevant libraries
import numpy as np
import pandas as pd

In [2]:
# create pandas from json files
teams_raw = pd.read_json('data/teams.json')
matches_raw = pd.read_json('data/matches_England.json')
events_raw = pd.read_json('data/events_England.json')
players_raw = pd.read_json('data/players.json')
ranks_raw = pd.read_json('data/player_rank.json')

## In order to make the dataset size more manageable we will focus on the English Premier League. This includes 20 teams from the United Kingdom. The UK includes England, Wales, Northern Ireland, and Scotland.

In [3]:
# copy teams_raw to teams to start editing
teams = teams_raw.copy()

# rename some columns for clarity
teams.rename(columns={"wyId": "teamId"}, inplace=True)

# add country column to teams dataframe
teams['country'] = teams.area.map(lambda v:v['name'])
teams['countryId'] = teams.area.map(lambda v:v['id'])
teams['countryId'] = teams.countryId.astype('int64')

# drop redundant column 'area'
teams.drop(['area'], axis=1, inplace=True)

# filter teams data frame to only include countries in the UK, countryID is 0
uk_teams = teams[(teams.countryId == 0)]

# filter out national teams by only including club
premier = uk_teams[uk_teams.type == 'club']

# sort by teamId
premier = premier.sort_values(['teamId'], axis=0, ascending=True)

# reset index to start from 0
premier = premier.reset_index(drop=True)

# export cleaned dataframe to csv
premier.to_csv('data/premier_league_teams.csv', index=False, header=True)

## Now lets make some adjustments to the players data frame and then filter out players who aren't in the premier league

In [4]:
# copy players_raw to teams to start editing
players = players_raw.copy()

# rename some columns for clarity
players.rename(columns={"wyId": "playerId"}, inplace=True)

# add position column to players dataframe
players['position'] = players.role.map(lambda v:v['code2'])

# convert currentTeamId to numeric, and coerce 'null' strings to Nan
players['currentTeamId'] = pd.to_numeric(players.currentTeamId, errors='coerce')
# convert birthDate to datetime object
players['birthDate'] = pd.to_datetime(players.birthDate)
# replace 0 values in height and weight with Nan
players['weight'] = players.weight.replace(0, np.nan)
players['height'] = players.height.replace(0, np.nan)
players['foot'] = players.foot.replace(['', 'null'], np.nan)

# drop unnecessary columns
players.drop(['passportArea', 'middleName', 'role', 'birthArea', 'currentNationalTeamId'], axis=1, inplace=True)

# make list of uk teams
uk_teams = premier.teamId.unique()
# filter only uk players
uk_players = players[players.currentTeamId.isin(uk_teams)]

# Kyle Taylor shows as the only player with missing data now so lets fill it in manually
# with data found here: https://www.fifaindex.com/player/239676/kyle-taylor/fifa20/
# get the index for Kyle Taylor
KT_idx = uk_players.index[(uk_players.firstName == 'Kyle') & (uk_players.lastName == 'Taylor')][0]
uk_players.loc[KT_idx,['weight', 'height', 'foot']] = [70, 178, 'right']

# this removed players with currentTeamId = Nan so lets convert to integer
uk_players['currentTeamId'] = uk_players['currentTeamId'].astype('int64')

# reset index
uk_players = uk_players.reset_index(drop=True)

# export cleaned dataframe to csv
uk_players.to_csv('data/premier_league_players.csv', index=False, header=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Let's clean up the matches data frame. This was imported from a json which was already filtered to the premier league so no filtering is required here.

In [5]:
# select desired columns for matches dataframe
matches = matches_raw.loc[:,['wyId', 'teamsData', 'dateutc', 'winner']]

# rename some columns for clarity
matches.rename(columns={"wyId": "matchId"}, inplace=True)

# initialize lists for building series
team1_list = []
team1_home = []
team1_scores = []
team2_list = []
team2_scores = []

# iterate through each row in dataframe
for index, row in matches.iterrows():
    # find dict keys
    keys = list(row['teamsData'].keys())
    # assign keys to team 1 and team 2
    team1 = row['teamsData'][keys[0]]
    team2 = row['teamsData'][keys[1]]
    
    # build lists for each team of relevant info
    team1_list.append(team1['teamId'])
    team1_scores.append(team1['score'])
    team2_list.append(team2['teamId'])
    team2_scores.append(team2['score'])
    
    # determine home team
    if team1['side'] == 'home':
        team1_home.append(1)
    else:
        team1_home.append(0)

# create columns from lists generated above
matches['team1'] = team1_list
matches['team1_scores'] = team1_scores
matches['team1_home'] = team1_home
matches['team2'] = team2_list
matches['team2_scores'] = team2_scores

# remove redundant column
matches.drop(['teamsData'], axis=1, inplace=True)

# export cleaned dataframe to csv
matches.to_csv('data/premier_league_matches.csv', index=False, header=True)

## The player rank is already pretty clean so let's just filter out players who aren't in the premier league.

In [6]:
# copy ranks_raw to ranks to start editing
ranks = ranks_raw.copy()

# make list of uk players
players_list = uk_players.playerId.unique()
# filter only uk players
uk_ranks = ranks[ranks.playerId.isin(players_list)]

# sort by matchId
uk_ranks = uk_ranks.sort_values(['matchId'], axis=0, ascending=True)

# reset index to start from 0
uk_ranks = uk_ranks.reset_index(drop=True)

# export cleaned dataframe to csv
uk_ranks.to_csv('data/premier_league_ranks.csv', index=False, header=True)

## Let's clean up the events data set focusing on the tags and the xy positions.

In [7]:
# copy ranks_raw to ranks to start editing
events = events_raw.copy()

#split positions column into start and end position colums, then drop original column
events[['start_pos', 'end_pos']] = pd.DataFrame(events.positions.values.tolist(), index=events.index)

#drop rows where end pos is NaN. This occurs when there are certain fouls or protests
events.dropna(inplace=True)
#split start_pos and end_pos to start_x, start_y, end_x, end_y
events[['start_y', 'start_x']] = pd.DataFrame(events.start_pos.values.tolist(), index=events.index)
events[['end_y', 'end_x']] = pd.DataFrame(events.end_pos.values.tolist(), index=events.index)

#remove columns which are no longer needed
events.drop(['positions', 'start_pos', 'end_pos'], axis=1, inplace=True)

# create new tags column with list instead of dictionary
tag_list = []
for index, row in events.iterrows():
    tag_list.append([d['id'] for d in row['tags']])

# overwrite the tags column with list
events['tags'] = tag_list

# export cleaned dataframe to csv
events.to_csv('data/premier_league_events.csv', index=False, header=True)